In [1]:
!pip install --user --upgrade catboost
!pip install --user --upgrade ipywidgets
!pip install shap
!pip install scikit-learn
!pip install --upgrade numpy
!jupyter nbextension enable --py widgetsnbextension

Defaulting to user installation because normal site-packages is not writeable
  Using cached numpy-1.23.5-cp310-cp310-win_amd64.whl (14.6 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.3
    Uninstalling numpy-1.26.3:
      Successfully uninstalled numpy-1.26.3


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.0 requires FuzzyTM>=0.4.0, which is not installed.


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
  Using cached numpy-1.26.3-cp310-cp310-win_amd64.whl (15.8 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.0 requires FuzzyTM>=0.4.0, which is not installed.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.26.3 which is incompatible.
Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


In [1]:
import catboost
print(catboost.__version__)
!python --version

1.2.2
Python 3.10.9


In [2]:
import pandas as pd
import os
import numpy as np
np.set_printoptions(precision=4)
import catboost
from catboost import *
from catboost import datasets

In [3]:
train = pd.read_csv('train2.csv', index_col=0)

In [4]:
test = pd.read_csv('test2.csv', index_col=0)

In [5]:
train=train.drop(['ts','gate_id', 'user_word'], axis=1)
test= test.drop(['ts','gate_id'], axis=1)
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37518 entries, 0 to 37517
Data columns (total 52 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   user_id    37518 non-null  float64
 1   gate_-1    37518 non-null  int64  
 2   gate_0     37518 non-null  int64  
 3   gate_1     37518 non-null  int64  
 4   gate_2     37518 non-null  int64  
 5   gate_3     37518 non-null  int64  
 6   gate_4     37518 non-null  int64  
 7   gate_5     37518 non-null  int64  
 8   gate_6     37518 non-null  int64  
 9   gate_7     37518 non-null  int64  
 10  gate_8     37518 non-null  int64  
 11  gate_9     37518 non-null  int64  
 12  gate_10    37518 non-null  int64  
 13  gate_11    37518 non-null  int64  
 14  gate_12    37518 non-null  int64  
 15  gate_13    37518 non-null  int64  
 16  gate_14    37518 non-null  int64  
 17  gate_15    37518 non-null  int64  
 18  gate_16    37518 non-null  int64  
 19  Monday     37518 non-null  int64  
 20  Tuesda

In [6]:
y = train.user_id
X = train.drop(['user_id'], axis=1)

In [7]:
cat_features = list(range(0, X.shape[1]))
print(cat_features)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50]


In [8]:
print('Labels: {}'.format(set(y)))
print('Zero count = {}, One count = {}'.format(len(y) - sum(y), sum(y)))

Labels: {0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 14.0, 15.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 38.0, 39.0, 40.0, 41.0, 42.0, 43.0, 44.0, 45.0, 46.0, 47.0, 48.0, 49.0, 50.0, 51.0, 52.0, 53.0, 54.0, 55.0, 56.0, 57.0}
Zero count = -1021203.0, One count = 1058721.0


In [9]:
dataset_dir = './turniket'
if not os.path.exists(dataset_dir):
    os.makedirs(dataset_dir)


In [10]:
train.to_csv(
    os.path.join(dataset_dir, 'train.csv'),
    index=False, sep=',', header=True)
test.to_csv(
    os.path.join(dataset_dir, 'test.csv'),
    index=False, sep=',', header=True)

In [11]:
!head turniket/train.csv

"head" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.


In [12]:
from catboost.utils import create_cd
feature_names = dict()
for column, name in enumerate(train):
    if column == 0:
        continue
    feature_names[column] = name
    
create_cd(
    label=0, 
    cat_features=list(range(1, train.columns.shape[0])),
    feature_names=feature_names,
    output_path=os.path.join(dataset_dir, 'train.cd')
)

In [13]:
!cat turniket/train.cd

"cat" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.


In [14]:
pool = Pool(
    data=os.path.join(dataset_dir, 'train.csv'), 
    delimiter=',', 
    column_description=os.path.join(dataset_dir, 'train.cd'),
    has_header=True)

In [15]:
print('Dataset shape')
print('dataset:' + str(pool.shape))

print('\n')
print('Column names')
print('dataset:')
print(pool.get_feature_names()) 


Dataset shape
dataset:(37518, 51)


Column names
dataset:
['gate_-1', 'gate_0', 'gate_1', 'gate_2', 'gate_3', 'gate_4', 'gate_5', 'gate_6', 'gate_7', 'gate_8', 'gate_9', 'gate_10', 'gate_11', 'gate_12', 'gate_13', 'gate_14', 'gate_15', 'gate_16', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday', 'hour', 'hour_0', 'hour_1', 'hour_3', 'hour_6', 'hour_7', 'hour_8', 'hour_9', 'hour_10', 'hour_11', 'hour_12', 'hour_13', 'hour_14', 'hour_15', 'hour_16', 'hour_17', 'hour_18', 'hour_19', 'hour_20', 'hour_21', 'hour_22', 'hour_23', 'min', 'day', 'month', 'dayofweek']


In [16]:
from sklearn.model_selection import train_test_split
X_train, X_validation, y_train, y_validation = train_test_split(X, y, train_size=0.8, random_state=1234)

In [17]:
from catboost import CatBoostClassifier
model = CatBoostClassifier(
    iterations=5,
    learning_rate=0.1,
    loss_function='MultiClass'
)
model.fit(
    X_train, y_train,
    cat_features=cat_features,
    eval_set=(X_validation, y_validation),
    verbose=False
)
print('Model is fitted: ' + str(model.is_fitted()))
print('Model params:')
print(model.get_params())

Model is fitted: True
Model params:
{'iterations': 5, 'learning_rate': 0.1, 'loss_function': 'MultiClass'}


In [18]:
from catboost import CatBoostClassifier
model = CatBoostClassifier(
    iterations=15,
loss_function='MultiClass'
)
model.fit(
    X_train, y_train,
    cat_features=cat_features,
    eval_set=(X_validation, y_validation),
)

Learning rate set to 0.5
0:	learn: 3.5441511	test: 3.5399247	best: 3.5399247 (0)	total: 3.69s	remaining: 51.7s
1:	learn: 3.3525562	test: 3.3614185	best: 3.3614185 (1)	total: 7.31s	remaining: 47.5s
2:	learn: 3.2294544	test: 3.2454537	best: 3.2454537 (2)	total: 10.9s	remaining: 43.5s
3:	learn: 3.1417029	test: 3.1614126	best: 3.1614126 (3)	total: 14.4s	remaining: 39.6s
4:	learn: 3.0873389	test: 3.1088025	best: 3.1088025 (4)	total: 17.9s	remaining: 35.9s
5:	learn: 3.0288211	test: 3.0512942	best: 3.0512942 (5)	total: 21.4s	remaining: 32.2s
6:	learn: 2.9795227	test: 3.0153984	best: 3.0153984 (6)	total: 24.9s	remaining: 28.5s
7:	learn: 2.9527073	test: 2.9949792	best: 2.9949792 (7)	total: 28.5s	remaining: 24.9s
8:	learn: 2.9380411	test: 2.9837314	best: 2.9837314 (8)	total: 32s	remaining: 21.4s
9:	learn: 2.9118734	test: 2.9616473	best: 2.9616473 (9)	total: 35.5s	remaining: 17.8s
10:	learn: 2.9070318	test: 2.9580154	best: 2.9580154 (10)	total: 39s	remaining: 14.2s
11:	learn: 2.8960524	test: 2.94

In [20]:
from catboost import CatBoostClassifier
model = CatBoostClassifier(
    iterations=50,
    random_seed=63,
    learning_rate=0.5,
    loss_function='MultiClass',
    custom_loss=['AUC', 'Accuracy']
)
model.fit(
    X_train, y_train,
    cat_features=cat_features,
    eval_set=(X_validation, y_validation),
    verbose=False,
    plot=True
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [25]:
model1 = CatBoostClassifier(
    learning_rate=0.7,
    iterations=30,
    random_seed=0,
    loss_function='MultiClass',
    train_dir='learing_rate_0.7'
)

model2 = CatBoostClassifier(
    learning_rate=0.01,
    iterations=30,
    random_seed=0,
    loss_function='MultiClass',
    train_dir='learing_rate_0.01'
)
model1.fit(
    X_train, y_train,
    eval_set=(X_validation, y_validation),
    cat_features=cat_features,
    verbose=False
)
model2.fit(
    X_train, y_train,
    eval_set=(X_validation, y_validation),
    cat_features=cat_features,
    verbose=False
)

In [24]:
from catboost import MetricVisualizer
MetricVisualizer(['learing_rate_0.01', 'learing_rate_0.7']).start()

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))